In [1]:
import pandas as pd
import numpy as np
seed = np.random.seed(42)
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="ticks", color_codes=True)

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',50)

%matplotlib inline
best_model = None
best_score = 0
hospital_name = 'host2'

In [2]:
data = pd.read_csv('../data/hospital' + str(2) + '_data.csv')

In [3]:
from sklearn.preprocessing import Imputer,StandardScaler,LabelEncoder
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE

/Users/refikcanmalli/anaconda3/envs/vodafone/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
data_e = data[data['IPG'] != 1]
data_e = data_e.drop('IPG', axis = 1);
data_e = data_e.drop('sbjBeatConsidered', axis = 1);
data_e = data_e.drop('numRRaveraged', axis = 1);
data_e = data_e.drop('Soggetti', axis = 1);
# df_columns = data.columns.tolist()
# matching_columns1 = [s for s in df_columns if "Parea" in s]
# matching_columns2 = [s for s in df_columns if "Pdur" in s]
# mc = matching_columns1 + matching_columns2
# data_e = data_e.dropna(subset=mc)
data_e['PCneg'].fillna(0, inplace=True)
data_e['AFclass'].replace('persistierend (>7 Tage, EKV)', 1, inplace=True)
data_e['AFclass'].replace('paroxysmal', 0, inplace=True)
categorical_variables_mask = data_e.dtypes==object
categorical_variables = data_e.columns[categorical_variables_mask]
for v in categorical_variables:
    data_e[v] = LabelEncoder().fit_transform(data_e[v])
del categorical_variables_mask,categorical_variables
data_e = pd.DataFrame(data=MICE(n_imputations=100, impute_type='col', n_nearest_columns=5).complete(data_e.as_matrix()), columns=data_e.columns, index=data_e.index);
y_df = data_e['AFclass'].copy()
x_df = data_e.drop('AFclass',axis=1, inplace=False)

[MICE] Completing matrix with shape (274, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.003
[MICE] Starting imputation round 2/110, elapsed time 0.065
[MICE] Starting imputation round 3/110, elapsed time 0.120
[MICE] Starting imputation round 4/110, elapsed time 0.166
[MICE] Starting imputation round 5/110, elapsed time 0.243
[MICE] Starting imputation round 6/110, elapsed time 0.299
[MICE] Starting imputation round 7/110, elapsed time 0.370
[MICE] Starting imputation round 8/110, elapsed time 0.412
[MICE] Starting imputation round 9/110, elapsed time 0.461
[MICE] Starting imputation round 10/110, elapsed time 0.530
[MICE] Starting imputation round 11/110, elapsed time 0.583
[MICE] Starting imputation round 12/110, elapsed time 0.638
[MICE] Starting imputation round 13/110, elapsed time 0.710
[MICE] Starting imputation round 14/110, elapsed time 0.760
[MICE] Starting imputation round 15/110, elapsed time 0.812
[MICE] Starting imputation round 16/110, elapsed time 0.859
[MI

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier
#from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

#f1_scorer = make_scorer(f1_score, average='macro',labels=Y.unique().tolist())
#myscorer = f1_scorer
myscorer = 'accuracy'

In [6]:
balancer = SMOTE(random_state=42)

In [7]:
def show_f_importance(estimator,train_data):
    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")

    for f in range(train_data.shape[1]):
        print("%d. feature %d - %s (%f)" % (f + 1, indices[f],train_data.columns[indices[f]], importances[indices[f]]))


In [8]:
param_grid = {"clf__strategy": ['most_frequent','stratified','prior','uniform']}

dummy = DummyClassifier(random_state=42)
plist = []
plist.append(('balancer', balancer))
plist.append(('clf', dummy))
pipeline = Pipeline(plist)

gs_dummy = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [9]:
gs_dummy.fit(x_df, y_df);

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.2s finished


In [10]:
gs_dummy.best_params_

{'clf__strategy': 'most_frequent'}

In [11]:
gs_dummy.best_score_

0.6824817518248175

In [12]:
param_grid = {"clf__max_depth": [10,30,50],
              "clf__criterion": ['gini', 'entropy'],
              "clf__n_estimators": [100,200,500,1000],
              "clf__random_state": [42],
              "clf__class_weight" : [None, 'balanced']}
# param_grid = {"classifier__max_depth": [30],
#               "classifier__n_estimators": [1000]}


rf = RandomForestClassifier()

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', rf))
pipeline = Pipeline(plist)
gs_rf = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [13]:
gs_rf.fit(x_df, y_df);

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   45.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  5.8min finished


In [14]:
gs_rf.best_params_

{'clf__class_weight': None,
 'clf__criterion': 'entropy',
 'clf__max_depth': 30,
 'clf__n_estimators': 100,
 'clf__random_state': 42}

In [15]:
gs_rf.best_score_


0.6277372262773723

In [16]:
if gs_rf.best_score_ > best_score:
    best_model = gs_rf
    best_score = gs_rf.best_score_

In [17]:
show_f_importance(gs_rf.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 4 - QRS (0.029896)
2. feature 45 - Parea_ECG_V8 (0.028055)
3. feature 3 - QTn (0.025425)
4. feature 1 - QTm_old (0.025402)
5. feature 25 - Pdur_ECG_V4 (0.024329)
6. feature 23 - Pdur_ECG_V3 (0.022998)
7. feature 34 - Parea_ECG_I (0.022602)
8. feature 40 - Parea_ECG_V3R (0.021703)
9. feature 55 - Pfwhm_ECG_V3 (0.019480)
10. feature 35 - Parea_ECG_II (0.019041)
11. feature 5 - PQ (0.018941)
12. feature 46 - Parea_ECG_V9 (0.018872)
13. feature 49 - Parea_ECG_aVR (0.018794)
14. feature 42 - Parea_ECG_V4R (0.018009)
15. feature 44 - Parea_ECG_V6 (0.017980)
16. feature 0 - RR (0.017346)
17. feature 37 - Parea_ECG_V1 (0.017156)
18. feature 62 - Pfwhm_ECG_V9 (0.017130)
19. feature 63 - Pfwhm_ECG_aVF (0.016525)
20. feature 12 - Pdur16mean (0.016523)
21. feature 67 - Age (0.016453)
22. feature 10 - PCarea (0.016333)
23. feature 54 - Pfwhm_ECG_V2 (0.016133)
24. feature 2 - QR (0.015876)
25. feature 51 - Pfwhm_ECG_II (0.015666)
26. feature 65 - Pfwhm_ECG_aVR (0.015259)


In [18]:
param_grid = {"clf__max_depth": [3,5,8],
              "clf__n_estimators": [100,200,500,1000],
              "clf__min_samples_leaf": [1,3,5,10]}
# param_grid = {"classifier__n_estimators": [50],
#               "classifier__max_depth": [5],
# 

gb = GradientBoostingClassifier(random_state=42)
plist = []
plist.append(('balancer', balancer))
plist.append(('clf', gb))
pipeline = Pipeline(plist)
gs_gb = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1,  scoring=myscorer, n_jobs=-1)

In [19]:
gs_gb.fit(x_df, y_df);

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   40.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  4.0min finished


In [20]:
gs_gb.best_params_

{'clf__max_depth': 5, 'clf__min_samples_leaf': 1, 'clf__n_estimators': 200}

In [21]:
gs_gb.best_score_

0.6605839416058394

In [22]:
if gs_gb.best_score_ > best_score:
    best_model = gs_gb
    best_score = gs_gb.best_score_

In [23]:
show_f_importance(gs_gb.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 1 - QTm_old (0.039163)
2. feature 37 - Parea_ECG_V1 (0.033949)
3. feature 34 - Parea_ECG_I (0.032819)
4. feature 50 - Pfwhm_ECG_I (0.032210)
5. feature 52 - Pfwhm_ECG_III (0.031865)
6. feature 0 - RR (0.030842)
7. feature 64 - Pfwhm_ECG_aVL (0.030241)
8. feature 2 - QR (0.030006)
9. feature 54 - Pfwhm_ECG_V2 (0.029419)
10. feature 47 - Parea_ECG_aVF (0.027212)
11. feature 40 - Parea_ECG_V3R (0.025544)
12. feature 4 - QRS (0.025515)
13. feature 3 - QTn (0.025276)
14. feature 39 - Parea_ECG_V3 (0.024238)
15. feature 45 - Parea_ECG_V8 (0.023873)
16. feature 67 - Age (0.023791)
17. feature 38 - Parea_ECG_V2 (0.022386)
18. feature 16 - Pdur16std (0.019062)
19. feature 10 - PCarea (0.018582)
20. feature 46 - Parea_ECG_V9 (0.018505)
21. feature 35 - Parea_ECG_II (0.018100)
22. feature 7 - PCneg (0.017898)
23. feature 59 - Pfwhm_ECG_V5 (0.017763)
24. feature 58 - Pfwhm_ECG_V4R (0.017143)
25. feature 25 - Pdur_ECG_V4 (0.017124)
26. feature 53 - Pfwhm_ECG_V1 (0.016678

In [24]:
param_grid = {"clf__learning_rate": [0.1,0.3,0.7,0.8],
              "clf__n_estimators": [100,1000,2000,2500,3000,5000]}

ab = AdaBoostClassifier(random_state=42)

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', ab))
pipeline = Pipeline(plist)

gs_ab = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [25]:
gs_ab.fit(x_df, y_df);

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 21.8min finished


In [26]:
gs_ab.best_params_

{'clf__learning_rate': 0.8, 'clf__n_estimators': 100}

In [27]:
gs_ab.best_score_

0.635036496350365

In [28]:

if gs_ab.best_score_ > best_score:
    best_model = gs_ab
    best_score = gs_ab.best_score_

In [29]:
show_f_importance(gs_ab.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 35 - Parea_ECG_II (0.070000)
2. feature 34 - Parea_ECG_I (0.050000)
3. feature 10 - PCarea (0.050000)
4. feature 64 - Pfwhm_ECG_aVL (0.050000)
5. feature 40 - Parea_ECG_V3R (0.050000)
6. feature 45 - Parea_ECG_V8 (0.050000)
7. feature 37 - Parea_ECG_V1 (0.040000)
8. feature 62 - Pfwhm_ECG_V9 (0.040000)
9. feature 59 - Pfwhm_ECG_V5 (0.040000)
10. feature 39 - Parea_ECG_V3 (0.030000)
11. feature 25 - Pdur_ECG_V4 (0.030000)
12. feature 50 - Pfwhm_ECG_I (0.030000)
13. feature 1 - QTm_old (0.030000)
14. feature 2 - QR (0.030000)
15. feature 61 - Pfwhm_ECG_V8 (0.030000)
16. feature 55 - Pfwhm_ECG_V3 (0.020000)
17. feature 54 - Pfwhm_ECG_V2 (0.020000)
18. feature 32 - Pdur_ECG_aVL (0.020000)
19. feature 30 - Pdur_ECG_V9 (0.020000)
20. feature 3 - QTn (0.020000)
21. feature 42 - Parea_ECG_V4R (0.020000)
22. feature 44 - Parea_ECG_V6 (0.020000)
23. feature 57 - Pfwhm_ECG_V4 (0.020000)
24. feature 47 - Parea_ECG_aVF (0.020000)
25. feature 23 - Pdur_ECG_V3 (0.020000)
2

In [30]:
param_grid = {"clf__max_depth": [3,5,8,10,15,30],
              "clf__n_estimators": [100,200,500,1000,1500],
              "clf__min_samples_leaf": [1,3,5,10]}

et = ExtraTreesClassifier(random_state=42)

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', et))
pipeline = Pipeline(plist)

gs_et = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1,  scoring=myscorer, n_jobs=-1)

In [31]:
gs_et.fit(x_df, y_df);

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   43.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 12.0min finished


In [32]:
gs_et.best_params_

{'clf__max_depth': 8, 'clf__min_samples_leaf': 1, 'clf__n_estimators': 100}

In [33]:
gs_et.best_score_

0.635036496350365

In [34]:
if gs_et.best_score_ > best_score:
    best_model = gs_et
    best_score = gs_et.best_score_

In [35]:
show_f_importance(gs_et.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 1 - QTm_old (0.032112)
2. feature 45 - Parea_ECG_V8 (0.028103)
3. feature 25 - Pdur_ECG_V4 (0.024425)
4. feature 23 - Pdur_ECG_V3 (0.022651)
5. feature 34 - Parea_ECG_I (0.021396)
6. feature 40 - Parea_ECG_V3R (0.020291)
7. feature 3 - QTn (0.019453)
8. feature 0 - RR (0.018263)
9. feature 12 - Pdur16mean (0.017997)
10. feature 18 - Pdur_ECG_I (0.017586)
11. feature 55 - Pfwhm_ECG_V3 (0.017145)
12. feature 42 - Parea_ECG_V4R (0.016893)
13. feature 65 - Pfwhm_ECG_aVR (0.016867)
14. feature 4 - QRS (0.016552)
15. feature 11 - P_LoffEon (0.016088)
16. feature 64 - Pfwhm_ECG_aVL (0.015925)
17. feature 38 - Parea_ECG_V2 (0.015906)
18. feature 66 - patsex (0.015845)
19. feature 8 - PCdur (0.015764)
20. feature 37 - Parea_ECG_V1 (0.015761)
21. feature 46 - Parea_ECG_V9 (0.015641)
22. feature 48 - Parea_ECG_aVL (0.015584)
23. feature 51 - Pfwhm_ECG_II (0.015534)
24. feature 28 - Pdur_ECG_V6 (0.015372)
25. feature 39 - Parea_ECG_V3 (0.015268)
26. feature 59 - Pfwhm_E

In [36]:
cdata = pd.read_csv('../data/central' +'_data.csv')
cdata.shape

(275, 73)

In [37]:
data_e = cdata[cdata['IPG'] != 1]
data_e = data_e.drop('IPG', axis = 1);
data_e = data_e.drop('sbjBeatConsidered', axis = 1);
data_e = data_e.drop('numRRaveraged', axis = 1);
data_e = data_e.drop('Soggetti', axis = 1);
# df_columns = data.columns.tolist()
# matching_columns1 = [s for s in df_columns if "Parea" in s]
# matching_columns2 = [s for s in df_columns if "Pdur" in s]
# mc = matching_columns1 + matching_columns2
# data_e = data_e.dropna(subset=mc)
data_e['PCneg'].fillna(0, inplace=True)
data_e['AFclass'].replace('persistierend (>7 Tage, EKV)', 1, inplace=True)
data_e['AFclass'].replace('paroxysmal', 0, inplace=True)
categorical_variables_mask = data_e.dtypes==object
categorical_variables = data_e.columns[categorical_variables_mask]
for v in categorical_variables:
    data_e[v] = LabelEncoder().fit_transform(data_e[v])
del categorical_variables_mask,categorical_variables
data_e = pd.DataFrame(data=MICE(n_imputations=100, impute_type='col', n_nearest_columns=5).complete(data_e.as_matrix()), columns=data_e.columns, index=data_e.index);
cy_df = data_e['AFclass'].copy()
cx_df = data_e.drop('AFclass',axis=1, inplace=False)

[MICE] Completing matrix with shape (270, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.003
[MICE] Starting imputation round 2/110, elapsed time 0.045
[MICE] Starting imputation round 3/110, elapsed time 0.091
[MICE] Starting imputation round 4/110, elapsed time 0.143
[MICE] Starting imputation round 5/110, elapsed time 0.195
[MICE] Starting imputation round 6/110, elapsed time 0.243
[MICE] Starting imputation round 7/110, elapsed time 0.288
[MICE] Starting imputation round 8/110, elapsed time 0.342
[MICE] Starting imputation round 9/110, elapsed time 0.386
[MICE] Starting imputation round 10/110, elapsed time 0.432
[MICE] Starting imputation round 11/110, elapsed time 0.496
[MICE] Starting imputation round 12/110, elapsed time 0.541
[MICE] Starting imputation round 13/110, elapsed time 0.608
[MICE] Starting imputation round 14/110, elapsed time 0.654
[MICE] Starting imputation round 15/110, elapsed time 0.702
[MICE] Starting imputation round 16/110, elapsed time 0.746
[MI

In [38]:
from sklearn.metrics import accuracy_score
p = best_model.predict(cx_df);
scores = pd.DataFrame({hospital_name: p})
scores.to_csv('../data/' + hospital_name +'_predictions.csv', index=False)
np.save('../data/' + hospital_name +'_acc',best_score)
accuracy_score(cy_df, p)

0.6111111111111112

In [39]:
best_score

0.6605839416058394